In [1]:
import pandas as pd
import numpy as np
from scipy.stats import norm

We're using Newyown's method to find the root of our function.  My orignal video on implied volatility and root finding is here,
https://youtu.be/Jpy3iCsijIU

Please check it out if you are unfamiliar with either.

In [2]:
def newtons_method(f, fprime, R = 0, max_iter = 1000, tol=1e-3, args = [], debug = False):
    count = 0
    epsilon = 1
    f_return = []
    fprime_return = []
    
    while epsilon >= tol:
        count += 1
        if count >= max_iter:
            print('Exiting on runaway loop.')
            return (R, count)
        
        old_R = R
        
        function_value = f(R, args = args)
        function_derivative = fprime(R, args = args)
        ind = np.where(function_derivative <= 0)
        ind = ind[0]
       
        R = -function_value / function_derivative + R
        
        if ind.size > 0:
            R[ ind ] = R[ ind ] * 0.5 + R[ ind ]
            
        if np.isscalar(R):
            epsilon = np.abs( (R - old_R) /old_R )
        else:
            epsilon = np.linalg.norm( R - old_R, np.Inf)
        
        if debug == True:
            f_return.append(function_value)
            fprime_return.append(function_derivative)
        
    return R, count

In [3]:
def call_price(sigma, S, K, r, t):
    d1 = np.multiply( 1. / sigma * np.divide(1., np.sqrt(t)),
        np.log(S/K) + (r + sigma**2 / 2.) * t  )
    d2 = d1 - sigma * np.sqrt(t)
    
    C = np.multiply(S, norm.cdf(d1)) - \
        np.multiply(norm.cdf(d2) * K, np.exp(-r * t))
    return C

def put_price(sigma, S, K, r, t):
    d1 = np.multiply( 1. / sigma * np.divide(1., np.sqrt(t)),
        np.log(S/K) + (r + sigma**2 / 2.) * t  )
    d2 = d1 - sigma * np.sqrt(t)
    
    P = -np.multiply(S, norm.cdf(-d1)) + \
        np.multiply(norm.cdf(-d2) * K, np.exp(-r * t))
    return P

def call_objective_function(sigma, args):
    S = args[0]
    K = args[1]
    r = args[2]
    t = args[3]
    price = args[4]
    
    return call_price(sigma, S, K, r, t) - price

def put_objective_function(sigma, args):
    S = args[0]
    K = args[1]
    r = args[2]
    t = args[3]
    price = args[4]
    
    return put_price(sigma, S, K, r, t) - price

def calculate_vega(sigma, args):
    S = args[0]
    K = args[1]
    r = args[2]
    t = args[3]
    
    d1 = np.multiply( 1. / sigma * np.divide(1., np.sqrt(t)),
        np.log(S/K) + (r + sigma**2 / 2.) * t  )
    d2 = d1 - sigma * np.sqrt(t)
    
    return S * norm.pdf(d1) * np.sqrt(t)

    return np.where(f_value > 0)

This was recorded on 12/23/2020.  We're looking at the 01/19/2021 expirtion which is 29 days away at the time I am making this.  Here are screenshots of the option chain.

<img src="Indian_index_price.png" width="50%">

<img src="Indian_option_chain.png" width="50%">

In [4]:
S = 13601.10                    #  Index price
K = 14000                       #  Strike price
r = 0.0293                      #  Risk-free rate
t = 29 / 365                    #  Time until expiration
C0 = (132.05 + 141.50) / 2      #  Call price taken to be the midprice between the bid and ask

args = (S, K, r, t, C0)         #  These need to be packaged into a tuple for our Black Scholes and Newtons code

In [5]:
#  Solve the problem
sigma, count = newtons_method(call_objective_function, calculate_vega, 0.50, args = args)
print(sigma)

0.18262482202177446
